<a href="https://colab.research.google.com/github/tamayodb/CCRNFLRL/blob/main/Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q-learning

In [11]:
import random

# =============== Grid ===============
# S = start, . = free. # =  wall, * = dirt, (clean once)

GRID = [
    ['S', '.', '*', '#'],
    ['.', '.', '.', '.'],
    ['#', '*', '.', '.'],
]
ROWS, COLS = len(GRID), len(GRID[0])
ACTIONS = ['U', 'D', 'L', 'R']
ARROW = {'U': '^', 'D': 'v', 'L': '<', 'R': '>'}

# Rewards
STEP_REWARD = -1
CLEAN_REWARD = +5

# Q-Learning params (tweak for class demos)
SEED = 7
EPISODES = 3000
ALPHA = 0.10
GAMMA = 0.95
EPS_START  = 0.30
EPS_END = 0.05
MAX_STEPS = 80 # per training episode
DEMO_STEPS = 60 # cap for greedy demo

random.seed(SEED)

# =============== Locate Start and Dirt ===============

START = None
DIRT_LIST = []
for r in range (ROWS):
  for c in range (COLS):
    if GRID[r][c] == 'S':
      START = (r, c)
    elif GRID[r][c] == '*':
      DIRT_LIST.append((r, c))
DIRT_INDEX = {pos: i for i, pos in enumerate(DIRT_LIST)}
ALL_CLEAN_MASK = (1 << len(DIRT_LIST)) - 1

def in_bounds(r,c): return 0 <= r < ROWS and 0 <= c < COLS
def is_wall(r,c): return GRID [r][c] == '#'

# =============== Environment ===============

def step_env(state, action):
  """
  State = (row, col, cleaned_mask)
  Move: apply STEP_REWARD; if first time on a '*' add CLEAN_REWARD once.
  Returns: (new_state, reward, done)
  """
  r, c, cleaned_mask = state
  nr, nc = r, c
  if action == 'U': nr -= 1
  elif action == 'D': nr += 1
  elif action == 'L': nc -= 1
  elif action == 'R': nc += 1

  # Blocked -> stay
  if not in_bounds(nr, nc) or is_wall(nr, nc):
    nr, nc = r, c

  reward = STEP_REWARD
  mask = cleaned_mask # Initialize mask from state
  if (nr, nc) in DIRT_INDEX:
    bit = 1 << DIRT_INDEX[(nr, nc)]
    if (mask & bit) == 0:
      mask |= bit
      reward += CLEAN_REWARD

  done = (mask == ALL_CLEAN_MASK)
  return (nr, nc, mask), reward, done

# =============== Q-Learning Helpers ===============

def agrmax_q(Q, s):
  best_v = float('-inf')
  ties = []
  for a in ACTIONS:
    v = Q.get((s,a), 0.0)
    if v > best_v:
      best_v, ties= v, [a]
    elif abs(v - best_v) < 1e-12:
      ties.append(a)
  return random.choice(ties)

def epsilon_greedy(Q, s, eps):
  if random.random() < eps:
    return random.choice(ACTIONS), True
  return agrmax_q(Q, s), False

# =============== Train (Q-Table) ===============

def train_q_learning(episodes=EPISODES):
  Q = {}
  rewards = []

  def eps_schedule(ep):
    # Linear decay from EPS_START to EPS_END
    t = ep / max(1, episodes - 1)
    return EPS_START * (1 - t) + EPS_END * t

  for ep in range(episodes):
    s = (START[0], START[1], 0) # start with nothing cleaned
    total, steps, done = 0, 0, False
    eps = eps_schedule(ep)

    while not done and steps < MAX_STEPS:
      a, _ = epsilon_greedy(Q, s, eps)
      s2, r, done = step_env(s, a)
      total += r

      # Q(s,a)←Q(s,a)+α[r+γa′max​Q(s′,a′)−Q(s,a)]
      max_next = max([Q.get((s2, ap), 0.0) for ap in ACTIONS])
      td_target = r + (0 if done else GAMMA * max_next)
      old = Q.get((s,a), 0.0)
      Q[(s,a)] = old + ALPHA * (td_target - old)

      s = s2
      steps += 1

    rewards.append(total)
  return Q, rewards

# =============== Greedy Demo (Use Learned Q) ===============

def greedy_run(Q, max_steps=DEMO_STEPS):
  s = (START[0], START[1], 0)
  path = [s]
  actions = []
  total = 0
  for _ in range(max_steps):
    a = agrmax_q(Q, s)
    s, r, done = step_env(s, a)
    total += r
    path.append(s)
    actions.append(a)
    if done: break
  return path, actions, total, done

# =============== Pretty Print ===============

def show_grid(state, next_a=None):
  r0, c0, mask = state
  lines = []
  for r in range(ROWS):
    row = []
    for c in range(COLS):
      if (r,c) == (r0, c0):
        row.append('A' if next_a is None else ARROW[next_a])
      elif GRID[r][c] == '#':
        row.append('#')
      elif GRID[r][c] == '*':
        bit = 1 << DIRT_INDEX[(r,c)]
        row.append('*' if (mask & bit) == 0 else '.')
      else:
        row.append(GRID[r][c])
    lines.append(" ".join(row))
  print('\n'.join(lines))

# =============== Run ===============

if __name__ == "__main__":
    print("Training...")
    Q, rewards = train_q_learning()
    last50 = sum(rewards[-50:]) / max(1, len(rewards[-50:]))
    print(f"Training done. Last-50 avg reward: {last50:.2f}")

    path, actions, total, done = greedy_run(Q)
    print("\nGreedy run after training:")
    for i, s in enumerate(path[:-1]):
        a = actions[i] if i < len(actions) else None
        print(f"\nStep {i}: (action={a})")
        show_grid(s, next_a=a)

    print("\n--- Summary ---")
    print("Finished:", done)
    print("Steps:", len(actions))
    print("Total reward:", total)

# Optional: peek at Q-values for START
def q_row(Qtab, state):
    return {a: round(Qtab.get((state, a), 0.0), 2) for a in ACTIONS}

print("\nQ at START:", q_row(Q, (START[0], START[1], 0)))

Training...
Training done. Last-50 avg reward: 4.60

Greedy run after training:

Step 0: (action=R)
> . * #
. . . .
# * . .

Step 1: (action=R)
S > * #
. . . .
# * . .

Step 2: (action=D)
S . v #
. . . .
# * . .

Step 3: (action=L)
S . . #
. . < .
# * . .

Step 4: (action=D)
S . . #
. v . .
# * . .

--- Summary ---
Finished: True
Steps: 5
Total reward: 5

Q at START: {'U': 3.08, 'D': 1.93, 'L': 3.08, 'R': 4.3}


## Experiment Trials

In [13]:
# ================= Experiment Trials =================

TRIALS = {
    "A": {"EPS_START": 1.0, "EPS_END": 0.05, "ALPHA": 0.10},
    "B": {"EPS_START": 1.0, "EPS_END": 0.05, "ALPHA": 0.15},
    "C": {"EPS_START": 0.30, "EPS_END": 0.05, "ALPHA": 0.10},
}

if __name__ == "__main__":
    results = []

    for name, params in TRIALS.items():
        print(f"\n===== Trial {name} =====")
        EPS_START = params["EPS_START"]
        EPS_END   = params["EPS_END"]
        ALPHA     = params["ALPHA"]

        # Train
        Q, rewards = train_q_learning()
        last50 = sum(rewards[-50:]) / max(1, len(rewards[-50:]))

        # Greedy run
        path, actions, total, done = greedy_run(Q)

        print("\nGreedy run after training:")
        for i, s in enumerate(path[:-1]):
            a = actions[i] if i < len(actions) else None
            print(f"\nStep {i}: (action={a})")
            show_grid(s, next_a=a)

        results.append({
            "Trial": name,
            "Q at START": q_row(Q, (START[0], START[1], 0)),
            "Last-50 Avg Reward": round(last50, 2),
            "Finished?": done,
            "Steps": len(actions),
            "Total Reward": total
        })

    # Summary table
    print("\n================ Summary Table ================")
    print(f"{'Trial':<5} | {'Last-50 Avg Reward':<20} | {'Finished?':<10} | {'Steps':<6} | {'Total Reward':<12} | Q at START")
    print("-"*90)
    for row in results:
        print(f"{row['Trial']:<5} | {row['Last-50 Avg Reward']:<20} | {str(row['Finished?']):<10} | {row['Steps']:<6} | {row['Total Reward']:<12} | {row['Q at START']}")



===== Trial A =====

Greedy run after training:

Step 0: (action=R)
> . * #
. . . .
# * . .

Step 1: (action=R)
S > * #
. . . .
# * . .

Step 2: (action=D)
S . v #
. . . .
# * . .

Step 3: (action=L)
S . . #
. . < .
# * . .

Step 4: (action=D)
S . . #
. v . .
# * . .

===== Trial B =====

Greedy run after training:

Step 0: (action=R)
> . * #
. . . .
# * . .

Step 1: (action=R)
S > * #
. . . .
# * . .

Step 2: (action=D)
S . v #
. . . .
# * . .

Step 3: (action=L)
S . . #
. . < .
# * . .

Step 4: (action=D)
S . . #
. v . .
# * . .

===== Trial C =====

Greedy run after training:

Step 0: (action=R)
> . * #
. . . .
# * . .

Step 1: (action=R)
S > * #
. . . .
# * . .

Step 2: (action=L)
S . < #
. . . .
# * . .

Step 3: (action=D)
S v . #
. . . .
# * . .

Step 4: (action=D)
S . . #
. v . .
# * . .

================ Summary Table ================
Trial | Last-50 Avg Reward   | Finished?  | Steps  | Total Reward | Q at START
-----------------------------------------------------------------

## How ε (exploration) and α (learning rate) change learning

1. Which trial has the highest Last-50 avg reward? Cite the number.
- Trial B, with 4.84
---

2. From START, what is the first greedy move in A, B, and C? Use the largest Q to justify.
- In all three trials, the first greedy move is Right (R) because Q(R) = 4.3, which is the highest among the actions.
---

3. In Trial C, Down = 1.93 is much lower. What does that imply, and what one change to ε or α would you try to improve learning?
- It implies the agent learned that Down is not a good option. I’d try using a larger ε to encourage more exploration or increasing α to strengthen updates.
---

4. If you raise α from 0.10 → 0.20 (same episodes), what happens to the speed of learning and the stability of the Last-50 curve? Explain briefly.
- Raising α to 0.20 makes learning faster but less stable.
---

5. Two settings reach the same greedy total reward (+5), but one has a higher Last-50 avg. Which configuration (episodes/α) would you prefer in practice, and why?
- I’d prefer the one with the higher Last-50 avg reward because it shows more consistent performance.
